In [3]:
import pandas as pd, numpy as np
behavior = pd.read_csv('project_prob.csv')
operatorProbs = pd.read_csv('LSCPoutputFinal.csv')

In [4]:
behavior.columns

Index(['project_id', 'cheatingprob_dlpattern', 'cheatingprob_dlcancellation',
       'cheatingprob_ulcancellation', 'cheatingprob'],
      dtype='object')

In [5]:
users = pd.read_pickle('userData.pkl')
operators = pd.read_pickle('operatorDataTx.pkl')

In [6]:
#create an email--> wallet converter
operators = operators.reset_index(drop=True)
emails = []
wallets = []
for i in range(len(operators)):
    for x in operators.loc[i,'fullEmailList']:
        emails.append(x)
        wallets.append(operators.loc[i,'wallet'])
        
adict = {'email':emails,'wallet':wallets}
emailWalletConverter = pd.DataFrame(adict)
emailWalletConverter

,email,wallet
0,76ef56b4ee699b7f2dbb99ac6a57bfb3a3ad93708fce99...,0x000000000c7c9cbb8485c5e38c6c0da6a1017c1f
1,89552b294746d6acdd6b3a2b995a8821345ee4619262a5...,0x000000043dc3052d771845a71efc05b67f40abb4
2,56f91963066bd109a009280b3b57dff4adb3f764f1e5ac...,0x00083c3ae5730507fa39a90220df704c09cf124d
3,e12652e32d55cd3f70d07914957a6e6472a1f3a1524e4f...,0x000e4b8d275cbe6af5c01a73bab484282317d8bb
4,06c35c443545d93e6b325906e6a970d34b9d08694cae7c...,0x00101a0e98ecea02681a5447c3cab0c787d109f6
...,...,...
8447,72cf5949e16ea70391bb42221bd9d46b521be1f9f3709e...,0xffbfc143924950bbd4ac6b6b4879001b6252dc40
8448,20095b1502f03f9137242589a39faefb520e5c3cfe37f0...,0xffbfc143924950bbd4ac6b6b4879001b6252dc40
8449,1d33422710ad68667d8e87e3ad3d7f76b72c63c609bf10...,0xffcfa143fbbc303790b6e5458cc3e3b570cf5343
8450,010adf9f55ed568491f64ba50e6b696732d72714ae316e...,0xffe336267854a7133a99a26910564439e5d57610


In [7]:
projects = pd.read_csv('allProjects.csv')

In [8]:
allUsers = pd.read_csv('allUsers.csv')

In [9]:
#step 1: connect project to email_hash
projToEmail = allUsers[['id','email_hash']].merge(projects, right_on='user_id',left_on='id',how='right')

In [13]:
projToEmail=projToEmail[['project_id','user_id','email_hash']].merge(behavior, on='project_id',how='left')

In [18]:
userCheatingProb = projToEmail.groupby('user_id').agg({'cheatingprob':'max'}).reset_index()

In [20]:
userCheatingProb.rename(columns={'cheatingprob':'maxUserCheatingProb'}, inplace=True)

In [21]:
userCheatingProb = userCheatingProb.merge(projToEmail[['user_id','email_hash']],on='user_id',how='left')

In [121]:
userCheatingProb.columns

Index(['user_id', 'maxUserCheatingProb', 'email_hash'], dtype='object')

In [23]:
operatorProbs = operatorProbs.merge(emailWalletConverter, on='wallet',how='left')

In [25]:
operatorProbs.columns

Index(['Unnamed: 0', 'wallet', 'LSCP_score', 'avgMonthPay', 'prob', 'email'], dtype='object')

In [26]:
fullView = operatorProbs.merge(userCheatingProb, left_on='email',right_on='email_hash', how='inner')

In [28]:
fullView = fullView[['wallet', 'user_id','maxUserCheatingProb', 'prob','avgMonthPay']]

In [33]:
#We have 655 operator accounts that also have user accounts
len(fullView.wallet.unique())

655

In [73]:
#We have 694 user accounts that also have operator accounts (some some users have multiple operator accounts)
len(fullView.user_id.unique())

694

In [78]:
numOperatorAccts = fullView.groupby('user_id').agg({'wallet':'nunique','maxUserCheatingProb':'max'}).reset_index()

In [83]:
numOperatorAccts.rename(columns={'wallet':'numOperatorWallets'},inplace=True)

In [159]:
#We see that some users have multiple operator accounts
numOperatorAccts.numOperatorWallets.value_counts()

1    656
2     34
3      4
Name: numOperatorWallets, dtype: int64

In [84]:
numUserAccts = fullView.groupby('wallet').agg({'user_id':'nunique','maxUserCheatingProb':'max'})
numUserAccts.rename(columns={'user_id':'numUserAccts'},inplace=True)

In [86]:
numUserAccts.numUserAccts.value_counts()

1     602
2      38
3      10
4       4
12      1
Name: numUserAccts, dtype: int64

In [92]:
len(numUserAccts)

655

In [134]:
#finalView = fullView.merge(numUserAccts, on=['wallet','maxUserCheatingProb'], how='inner').drop_duplicates()
finalView = fullView.merge(numOperatorAccts, on=['user_id','maxUserCheatingProb'], how='inner').drop_duplicates().dropna()

In [135]:
finalView.isnull().sum()

wallet                 0
user_id                0
maxUserCheatingProb    0
prob                   0
avgMonthPay            0
numOperatorWallets     0
dtype: int64

In [136]:
len(finalView)

322

In [137]:
print(len(finalView.wallet.unique()),len(finalView.user_id.unique()))

292 302


In [142]:
pd.set_option('display.max_rows',400)
finalView = finalView.rename(columns={'prob':'operatorCheatingProb'})

In [143]:
finalView.columns

Index(['wallet', 'user_id', 'maxUserCheatingProb', 'operatorCheatingProb',
       'avgMonthPay', 'numOperatorWallets'],
      dtype='object')

**Here we have a list of all operator/user combinations with >.8 probability of cheating in each category**

In [150]:
finalView[(finalView.maxUserCheatingProb>.8)&(finalView.operatorCheatingProb>.8)]

,wallet,user_id,maxUserCheatingProb,operatorCheatingProb,avgMonthPay,numOperatorWallets
9,0x003dd7051bd7b10beb092df05258b5b847b0c8fc,87fe37e0b4354f0c89881508b6cbd4fd,0.864061,0.882376,5.337503,3
10,0x5789caf7b28d0e0869769c0e66388cb44747d47c,87fe37e0b4354f0c89881508b6cbd4fd,0.864061,0.974773,36.242408,3
60,0x11d484fab9c282d3854e1f85b8af62d7b686d795,c5ec32efcb734547ab233fd570bd8424,0.878435,0.897540,24.969701,1
381,0x6b21925ae3cc58c0abfc31792bf35dd42f0bb81d,aaa0bd2db2914b349504a20bc80b81bd,0.825816,0.927595,0.526446,2
382,0xfafc9af3c0353d95244a53a3fc80e22295bec8a0,aaa0bd2db2914b349504a20bc80b81bd,0.825816,0.970166,4.238282,2
390,0x6be35233115075a56bcb95cceda284b5bc3aeafe,4ba1ca9401d34adfb12bd36706df49b8,0.931086,0.845043,2.940931,1
456,0x7a582bcf950a30c0ca783b99026be55a6ad6efb6,62e1ed08dcad43dfb99a356ce7d6ceb2,0.962544,0.865999,19.641417,1


In [151]:
finalView.to_csv('user_operator_view.csv')

In [167]:
otherFactors = finalView[(finalView.maxUserCheatingProb>.8)&(finalView.operatorCheatingProb>.8)].merge(operators[['wallet','unique_nodes','numAssocEmails']],on='wallet',how='left')

In [168]:
otherFactors[['wallet','user_id','numAssocEmails']]

,wallet,user_id,numAssocEmails
0,0x003dd7051bd7b10beb092df05258b5b847b0c8fc,87fe37e0b4354f0c89881508b6cbd4fd,1.0
1,0x5789caf7b28d0e0869769c0e66388cb44747d47c,87fe37e0b4354f0c89881508b6cbd4fd,1.0
2,0x11d484fab9c282d3854e1f85b8af62d7b686d795,c5ec32efcb734547ab233fd570bd8424,8.0
3,0x6b21925ae3cc58c0abfc31792bf35dd42f0bb81d,aaa0bd2db2914b349504a20bc80b81bd,1.0
4,0xfafc9af3c0353d95244a53a3fc80e22295bec8a0,aaa0bd2db2914b349504a20bc80b81bd,1.0
5,0x6be35233115075a56bcb95cceda284b5bc3aeafe,4ba1ca9401d34adfb12bd36706df49b8,3.0
6,0x7a582bcf950a30c0ca783b99026be55a6ad6efb6,62e1ed08dcad43dfb99a356ce7d6ceb2,4.0


In [161]:
operators.columns

Index(['wallet', 'unique_nodes', 'minAuditReputation', 'avgAuditReputation',
       'maxAuditReputation', 'minFreeDisk', 'avgFreeDisk', 'maxFreeDisk',
       'minPieceCount', 'avgPieceCount', 'maxPieceCount',
       'earliestNodeCreated', 'mostRecentNodeCreated', 'fullWalletList',
       'fullEmailList', 'numAssocWallets', 'numAssocEmails', '2020-02-01',
       '2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01', '2020-07-01',
       '2020-08-01', '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01',
       '2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', 'type',
       'sixMonthPay', 'avgMonthPay', 'numOutbound', 'uniqueOutbound',
       'tokenTypes', 'numInbound', 'uniqueInbound', 'associatesTo',
       'associatesFrom', 'associates', 'numAssoc'],
      dtype='object')

# Stop here

**The following pulls in the 65 candidates surfaced by operator clustering and checks them against the above**

In [170]:
#Read in data on the 65 candidates
cand65 = pd.read_pickle('candidateDF2.pkl')

In [171]:
finalView.merge(cand65, left_on='wallet',right_on='wallet',how='right').dropna()

,wallet,user_id,maxUserCheatingProb,operatorCheatingProb,avgMonthPay_x,numOperatorWallets,unique_nodes,minAuditReputation,avgAuditReputation,maxAuditReputation,...,OCSVM,OCSVM_score,ISOFOREST,ISOFOREST_score,COPOD,COPOD_score,LSCP,LSCPadj,LSCP_score,prob
21,0x5789caf7b28d0e0869769c0e66388cb44747d47c,1b3f384cc2d44f3c9a25f3b7c06de905,0.264431,0.974773,36.242408,3.0,26.0,1.000000,1.000000,1.0,...,1,1.257391,1,0.001907,1,34.672967,1,1,1.934791,0.949834
22,0x5789caf7b28d0e0869769c0e66388cb44747d47c,87fe37e0b4354f0c89881508b6cbd4fd,0.864061,0.974773,36.242408,3.0,26.0,1.000000,1.000000,1.0,...,1,1.257391,1,0.001907,1,34.672967,1,1,1.934791,0.949834
23,0x5836d1c3bf3408451d010f0a042e25e987ebd889,c4168ddf68514d409a3a75db6e27ce1c,0.145408,0.959891,60.260527,1.0,38.0,0.573435,0.992657,1.0,...,0,1.001802,1,0.003747,1,34.714239,1,1,1.862019,0.944239
24,0x5a913899700b70dea2e5017462078a7173126d20,c333525f53db498fb552fdcd165907eb,0.003238,0.982629,12.267274,1.0,24.0,0.961311,0.999463,1.0,...,0,0.151349,1,0.003552,1,35.834687,1,1,1.266331,0.859252
27,0x74d3a02380785af7d4617562f7ffd5946129c409,35f124922b3c4e23a862e0ed24bc010f,0.505767,0.971104,14.326732,2.0,14.0,0.487179,0.975945,1.0,...,0,0.584833,0,-0.004647,1,33.576193,1,1,1.855156,0.943644
28,0x74d3a02380785af7d4617562f7ffd5946129c409,88770938a12b43cca220cbc7da4241a1,0.000000,0.971104,14.326732,2.0,14.0,0.487179,0.975945,1.0,...,0,0.584833,0,-0.004647,1,33.576193,1,1,1.855156,0.943644
29,0x74d3a02380785af7d4617562f7ffd5946129c409,b7888aa2922d4fd2bc484f4f913c5a12,0.701314,0.971104,14.326732,2.0,14.0,0.487179,0.975945,1.0,...,0,0.584833,0,-0.004647,1,33.576193,1,1,1.855156,0.943644
30,0x757898f2e297fd512dd647f3da9c8ab8adc6a382,386157c051b04293b315d64806192537,0.138457,0.989926,14.991272,2.0,67.0,0.487179,0.965158,1.0,...,1,1.159546,1,0.035004,1,36.776901,1,1,2.472369,0.972386
34,0x87b27781ce30ed5ff690e88eaedf3991277f4136,056b7737d5564f1aa3d72da3eba26ae2,0.530069,0.989704,13.277643,2.0,8.0,0.598737,0.949810,1.0,...,0,0.607877,1,0.018133,1,37.203651,1,1,2.602042,0.977590
35,0x8b51c0ecf0381c41e1b299ebfafa352997983741,560ce3b63f0248c38038a7729bc5fa4f,0.547578,0.960437,13.602709,1.0,25.0,0.598737,0.969262,1.0,...,1,1.119761,1,0.015155,0,31.377340,1,1,1.883920,0.946056
